# Análise dos Algoritmos de Fleury e Hierholzer

### Higor da Silva Camelo, Erick Gabriel Ferreira Gaspar, Pedro Henrique Santos Moreira


---
## Sumário

- Introdução
- Implementação dos Algoritmos
  - Algoritmo de Fleury
  - Algoritmo de Hierholzer
  - Funções para teste
    - make_random_graph
    - execute_with_timer
    - test_alg
- Execução dos Algoritmos
  - Fleury
  - Hierholzer
- Diferença Percentual
  - Base gráfica
  - Cálculos de diferença relativa - Algoritmo de Fleury
  - Cálculos de diferença relativa - Algoritmo de Hierholzer
- Conclusão

---
## Introdução

Nesse notebook, iremos analisar a performance de dois algoritmos de busca de ciclos Eulerianos. Mostraremos as implementações, funções auxiliares para testes e o cálculo da diferença relativa de cada um. No fim, concluiremos qual deles é o melhor algoritmo.

---
## Implementação dos Algoritmos

Primeiro, vamos importar as seguintes bibliotecas para a utilizarmos em nossos códigos.



In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import math
import time
import random
import sys


Agora, vamos definir as implementações para o algoritmo de Fluery e Hierholzer.

---
### Fleury

In [ ]:
# Função que verifica se a aresta u-v é uma ponte
def eh_ponte(G, u, v):
    # Remove a aresta u-v temporariamente
    G[u].remove(v)
    G[v].remove(u)

    # Vetor de visitados para a busca em profundidade (DFS)
    visitados = [False] * len(G)

    # Função interna de DFS (busca em profundidade)
    def dfs(v):
        visitados[v] = True
        # Percorre todos os vértices adjacentes de v
        for w in G[v]:
            if not visitados[w]:
                dfs(w)

    # Inicia a DFS a partir de u para verificar conectividade
    dfs(u)

    # Recoloca a aresta u-v (desfazendo a remoção temporária)
    G[u].append(v)
    G[v].append(u)

    # Se v não foi visitado, a aresta u-v era uma ponte (sua remoção desconecta o grafo)
    return not visitados[v]

# Função que implementa o algoritmo de Fleury para encontrar o ciclo Euleriano
def fleury(G, u):
    # Contagem total de arestas no grafo (somatório de todas as listas de adjacência dividido por 2)
    arestas = sum(len(v) for v in G.values()) // 2

    # Inicia o ciclo com o vértice u
    C = [u]

    # Enquanto houver arestas não percorridas
    while arestas > 0:
        # Percorre todos os vértices adjacentes a u
        for v in G[u][:]:
            # Verifica se a aresta u-v é uma ponte (evitando-a, se possível)
            if len(G[u]) > 1 and eh_ponte(G, u, v):
                continue  # Se for uma ponte e há outras opções, ignora esta aresta

            # Remove a aresta u-v do grafo (como parte do caminho)
            G[u].remove(v)
            G[v].remove(u)

            # Adiciona o próximo vértice v ao ciclo Euleriano
            C.append(v)

            # Move o vértice u para v e continua o ciclo
            u = v

            # Diminui a contagem de arestas, já que percorremos uma
            arestas -= 1

            # Quebra o loop assim que uma aresta válida é encontrada
            break

    # Retorna o ciclo Euleriano encontrado
    return C

O algorimto consiste na busca de um ciclo Euleriano partindo de um ponto inicial do grafo, passando por todas as suas arestas apenas uma vez e retornando para o início. Ele se baseia na lógica de não "queimar" pontes quando possível, as quais são arestas cuja a remoção implica na desconexão do grafo. Iremos remover cada aresta atravessada, o que nos impede de passarmos por ela novamente.


---
### Hierholzer

In [ ]:
# Função que implementa o algoritmo de Hierholzer para encontrar um ciclo Euleriano
def hierholzer(graph, start_vertex):
    # Calcula o grau de cada vértice (quantidade de arestas conectadas a ele)
    degree = {vertex: len(neighbors) for vertex, neighbors in graph.items()}

    # Pilha usada para construir o ciclo. Começa no vértice inicial (start_vertex)
    stack = [start_vertex]

    # Lista que vai armazenar o ciclo Euleriano final
    cycle = []

    # Enquanto a pilha não estiver vazia
    while stack:
        # Pega o vértice no topo da pilha, mas não o remove
        u = stack[-1]

        # Se o vértice ainda tiver arestas não percorridas (grau maior que 0)
        if degree[u] > 0:
            # Remove uma das arestas (u, v) de u
            v = graph[u].pop()

            # Também remove a aresta (v, u) de v (grafo não direcionado)
            graph[v].remove(u)

            # Atualiza os graus de u e v (já que uma aresta foi removida)
            degree[u] -= 1
            degree[v] -= 1

            # Adiciona o vértice v à pilha (continua o caminho a partir de v)
            stack.append(v)
        else:
            # Se não há mais arestas não percorridas de u, adiciona u ao ciclo
            # e remove u da pilha (retrocede no caminho)
            cycle.append(stack.pop())

    # O ciclo Euleriano foi construído em ordem reversa, então invertemos para obter a ordem correta
    return cycle[::-1]


A ideia central do algoritmo é buscar subciclos eulerianos em um grafo a partir de um vértice qualquer e os removendo a medida que são encontrados no mesmo instante que são adicionados no ciclo principal que ao final será o resultado da união dos subciclos.

---
### Funções para teste

Aqui definiremos as funções usadas para os testes de performance dos dois algoritmos.


#### make_random_graph

Essa função tem a responsabilidade de criar novos grafos a fim de iniciarmos nossos testes.

- Parâmetros: `tam` representa o número de vértices do grafo a ser criado.
- Saída: `adjacencias` representa o nosso grafo recém formado.

In [ ]:
# Função para gerar um grafo pseudo-aleatório com um número ímpar de vértices
def make_random_graph(tam):
    # Se o tamanho (número de vértices) for par, torna-o ímpar adicionando 1
    if tam % 2 == 0:
        tam = tam + 1

    # Cria um dicionário para armazenar a lista de adjacências de cada vértice
    adjacencias = {i: [] for i in range(tam)}

    # Percorre todos os vértices do grafo
    for i in range(tam):
        # Casos especiais para os vértices nas extremidades do grafo:

        if i == 0:
            # Vértice 0 se conecta aos vértices 1 e 2
            adjacencias[i].extend([1, 2])
        elif i == 1:
            # Vértice 1 se conecta aos vértices 0 e 2
            adjacencias[i].extend([0, 2])
        elif i == tam - 2:
            # Penúltimo vértice se conecta ao vértice anterior e ao último
            adjacencias[i].extend([i - 1, i + 1])
        elif i == tam - 1:
            # Último vértice se conecta aos dois vértices anteriores
            adjacencias[i].extend([i - 1, i - 2])

        # Para os demais vértices, que não estão nas extremidades:
        else:
            # Se o índice do vértice for par
            if i % 2 == 0:
                # Conecta-se a dois vértices anteriores e dois vértices posteriores
                adjacencias[i].extend([i - 2, i - 1, i + 1, i + 2])
            # Se o índice do vértice for ímpar
            else:
                # Conecta-se apenas ao vértice anterior e ao próximo
                adjacencias[i].extend([i - 1, i + 1])

    # Retorna o dicionário que representa o grafo com as listas de adjacências
    return adjacencias


Utilizamos `{i: [] for i in range(tam)}` para estruturarmos o grafo em uma lista de adjacências, dado que cada vértice é a `key` para um lista de outros vértices. Devemos garantir também que o número de vértices do grafo gerado é ímpar, para não comprometer a formação do ciclo Euleriano.

#### execute_with_timer

Essa função tem a responsabilidade de executar o nosso algoritmo e calcular o tempo de execução.

- Parâmetros: `func` representa o algoritmo a ser executado e `timer` o nosso tempo limite de execução.
- Saída: `delta`, a diferença entre o tempo de início e o tempo de finalização da execução.

In [ ]:
def execute_with_timer(func, timer):
    start = time.time()

    func()

    end = time.time()

    delta = end - start

    if delta > timer:
        return -1

    return delta

#### teste_alg

Essa função tem a responsabilidade de chamar as outras duas funções e salvar todas as entradas e tempo de execução do algoritmo.

- Parâmetros: `alg`, representa o algoritmo a ser executado. `timer`, o nosso tempo limite de execução.
- Saída: `input_sizes, execution_times`, representam, respectivamente, a lista de entradas do algoritmo e a lista de tempos de execução do algoritmo.

In [ ]:
# Função para testar o tempo de execução de um algoritmo com grafos de diferentes tamanhos
def teste_alg(alg, timer):
    # Tamanho inicial do grafo de entrada
    input_size = 2

    # Armazena o tempo de execução anterior (começando com None)
    last_time: float = None

    # Listas para armazenar os tempos de execução e os tamanhos de entrada
    execution_times = []
    input_sizes = []
    tempo_total = 0

    # Loop até que o tempo de execução retorne -1 (indicando que ultrapassou o limite)
    while last_time == None or last_time != -1:
        # Gera um grafo aleatório com o tamanho atual (input_size)
        graph = make_random_graph(input_size)

        # Cria uma função lambda que executa o algoritmo 'alg' com o grafo gerado, começando no vértice 0
        func = lambda: alg(graph, 0)

        # Mede o tempo de execução do algoritmo usando o timer fornecido
        alg_execution_time = execute_with_timer(func, timer)

        # Se o tempo de execução for válido (diferente de -1, que indica que o tempo foi muito alto)
        if alg_execution_time != -1:
            # Adiciona o tempo de execução à lista
            execution_times.append(alg_execution_time)
            # Adiciona o tamanho da entrada correspondente à lista
            input_sizes.append(input_size)

            tempo_total += alg_execution_time

            print(f"Input size: {input_size}, execution time: {alg_execution_time:.6f} segundos")

        # Atualiza a variável 'last_time' com o tempo de execução atual
        last_time = alg_execution_time

        # Dobra o tamanho de entrada para o próximo teste
        input_size *= 2

    # Retorna as listas de tamanhos de entrada e tempos de execução
    return input_sizes, execution_times


Iremos utilizar o método "razão dobrando" para a obtenção de entradas, visto que nosso algoritmo é polinomial. Dito isso, a cada fim de execução, dobraremos o valor de `input_size` para assim criar o próximo grafo na sequência. Por fim, quando `timer`for igual a -1, também definida na função anterior, significa que o tempo de execução com a entrada atual excede o nosso tempo limite, obrigando o fim da execução.

---
## Execução dos Algoritmos

Nesta seção, vamos executar os dois algoritmos.

---
### Algoritmo de Fleury

Iremos aumentar o limite de chamadas recursivas da linguagem, a fim de evitar problemas com a parada da execução por conta desse fator.


In [ ]:
    sys.setrecursionlimit(10000000)

    fle_sizes, fle_times = teste_alg(fleury, 100)

Input size: 2, execution time: 0.000023 segundos
Input size: 4, execution time: 0.000033 segundos
Input size: 8, execution time: 0.000066 segundos
Input size: 16, execution time: 0.000155 segundos
Input size: 32, execution time: 0.000473 segundos
Input size: 64, execution time: 0.001970 segundos
Input size: 128, execution time: 0.008639 segundos
Input size: 256, execution time: 0.027717 segundos
Input size: 512, execution time: 0.107806 segundos
Input size: 1024, execution time: 0.441007 segundos
Input size: 2048, execution time: 1.961805 segundos
Input size: 4096, execution time: 5.254186 segundos
Input size: 8192, execution time: 69.997123 segundos


- `fle_sizes, fle_times:` lista de entradas e de tempos de execução respectivamente.

Assim, teremos uma relação entre tamanho de entradas e tempo de execução como no exemplo a seguir:

```
Input size: 2, execution time: 0.000047 segundos
Input size: 4, execution time: 0.000035 segundos
Input size: 8, execution time: 0.000063 segundos
Input size: 16, execution time: 0.000157 segundos
Input size: 32, execution time: 0.000478 segundos
Input size: 64, execution time: 0.001807 segundos
Input size: 128, execution time: 0.003791 segundos
Input size: 256, execution time: 0.013986 segundos
Input size: 512, execution time: 0.054191 segundos
Input size: 1024, execution time: 0.234888 segundos
Input size: 2048, execution time: 1.082525 segundos
Input size: 4096, execution time: 4.573834 segundos
Input size: 8192, execution time: 66.721341 segundos
```

---
### Algoritmo de Hierholzer

In [ ]:
    h_sizes, h_times = teste_alg(hierholzer, 100)

Input size: 2, execution time: 0.000018 segundos
Input size: 4, execution time: 0.000020 segundos
Input size: 8, execution time: 0.000077 segundos
Input size: 16, execution time: 0.000049 segundos
Input size: 32, execution time: 0.000623 segundos
Input size: 64, execution time: 0.000146 segundos
Input size: 128, execution time: 0.000296 segundos
Input size: 256, execution time: 0.000654 segundos
Input size: 512, execution time: 0.000994 segundos
Input size: 1024, execution time: 0.001608 segundos
Input size: 2048, execution time: 0.003279 segundos
Input size: 4096, execution time: 0.007731 segundos
Input size: 8192, execution time: 0.014864 segundos
Input size: 16384, execution time: 0.028354 segundos
Input size: 32768, execution time: 0.054856 segundos
Input size: 65536, execution time: 0.204615 segundos
Input size: 131072, execution time: 0.428417 segundos
Input size: 262144, execution time: 0.834484 segundos
Input size: 524288, execution time: 0.876345 segundos
Input size: 1048576, 

- `h_sizes, h_times:` lista de entradas e de tempos de execução respectivamente.

Assim, teremos uma relação entre tamanho de entradas e tempo de execução como no exemplo a seguir:

```
Input size: 2, execution time: 0.000017 segundos
Input size: 4, execution time: 0.000026 segundos
Input size: 8, execution time: 0.000021 segundos
Input size: 16, execution time: 0.000035 segundos
Input size: 32, execution time: 0.000079 segundos
Input size: 64, execution time: 0.000179 segundos
Input size: 128, execution time: 0.000268 segundos
Input size: 256, execution time: 0.000610 segundos
Input size: 512, execution time: 0.003302 segundos
Input size: 1024, execution time: 0.002758 segundos
Input size: 2048, execution time: 0.009646 segundos
Input size: 4096, execution time: 0.011748 segundos
Input size: 8192, execution time: 0.023370 segundos
Input size: 16384, execution time: 0.052293 segundos
Input size: 32768, execution time: 0.125199 segundos
Input size: 65536, execution time: 0.234795 segundos
Input size: 131072, execution time: 0.450014 segundos
Input size: 262144, execution time: 0.470104 segundos
Input size: 524288, execution time: 0.935472 segundos
Input size: 1048576, execution time: 1.872805 segundos
Input size: 2097152, execution time: 5.278626 segundos
Input size: 4194304, execution time: 9.265792 segundos
Input size: 8388608, execution time: 16.660990 segundos
Input size: 16777216, execution time: 34.502066 segundos
Input size: 33554432, execution time: 68.361356 segundos
```

---
## Diferença Percentual

### Base gráfica


Utilizando a função `plot_graph`, iremos gerar o gráficos de entradas por tempos. Os parâmetros são: `input_sizes`, para a lista de entradas; `execution_times`, para a lista de tempos; `name`, para o nome do algoritmo em questão.

In [ ]:
def plot_graph(input_sizes, execution_times, name):
  lx = np.log2(np.array(input_sizes))
  ly = np.log2(np.array(execution_times))

  # Gráfico de dispersão e linha
  plt.scatter(lx, ly, label="Data points")
  plt.plot(lx, ly, '-o', label="Log-log plot")  # Conecta os pontos com uma linha
  plt.xlabel('log2(Input Sizes)')
  plt.ylabel('log2(Execution Times)')
  plt.title(f'Log-Log Plot - {name}')
  plt.legend()
  plt.show()

Agora, iremos chamar a função para os repectivos algorimtos.

In [ ]:
  plot_graph( fle_sizes, fle_times, "Fleury")

In [ ]:
  plot_graph( h_sizes, h_times, "Hierholzer")

### Cálculos de diferença relativa - Algoritmo de Fleury

#### 1. Grau do Polinômio \( d \):

$$
d = \frac{\log(\frac{66.721341}{4.573834})}{\log(2)} \approx 3.87
$$

#### 2. Constante \( c \):

$$
c = \frac{4.573834}{4096^{3.87}} \approx 4.00 \times 10^{-14}
$$

#### 3. Previsão para \( n = 8192 \):

$$
T(8192) = 4.00 \times 10^{-14} \cdot 8192^{3.87} \approx 45.31 \, \text{segundos}
$$

#### 4. Diferença Relativa:

$$
\text{Diferença} = 100 \times \frac{|66.721341 - 45.31|}{45.31} \approx 47.26\%
$$

Conclusão: A diferença entre o tempo medido e o previsto é de **47.26%**.

---

### Cálculos de diferença relativa - Algoritmo de Hierholzer

#### 1. Grau do Polinômio \( d \):

$$
d = \frac{\log(\frac{68.361356}{34.502066})}{\log(2)} = 0,98649826
$$

#### 2. Constante \( c \):

$$
c = \frac{34.502066}{16777216^{0,98649826}} = 0,000002574  = 2,574 \times 10^{-6}
$$

#### 3. Previsão para \( n = 33554432 \):

$$
T(33554432) = 2,574 \times 10^{-6} \cdot 33554432^{0,98649826 } = 68.351447443 ≈ 68.35 \text{ segundos}
$$

#### 4. Diferença Relativa:

$$
\text{Diferença} = 100 \times \frac{|68.361356 - 68.35|}{68.35} = 0,016614484 \approx 0,017\%
$$

Conclusão: A diferença entre o tempo medido e o previsto é de **0,017%** aproximadamente.


---
## Conclusão

#### Grau \(d\):
- **Fleury**: Com \(d \approx 3.87\), o crescimento do tempo de execução é quase quartico (\(O(n^4)\)), tornando-se ineficiente para entradas grandes.
- **Hierholzer**: Com \(d \approx 0.99\), o algoritmo cresce de forma quase linear (\(O(n)\)), sendo muito mais eficiente.

#### Diferença Relativa:
- **Fleury**: Diferença de **47.26%** entre o tempo previsto e o medido, sugerindo que o modelo preditivo é impreciso para grandes entradas.
- **Hierholzer**: Diferença de apenas **0.017%**, mostrando alta precisão no modelo preditivo.

#### Tempos e Tamanhos de Entrada:
- **Fleury**: Para \(n = 8192\), o tempo foi de **66.72s**, crescendo rapidamente conforme o tamanho aumenta.
- **Hierholzer**: Para \(n = 33554432\), o tempo foi de **68.36s**, obtendo números que reforçam sua eficiência mesmo para entradas grandes.

### Conclusão:
Hierholzer, com crescimento linear e previsível, é muito mais eficiente para grandes instâncias, enquanto Fleury, com crescimento quartico, é adequado apenas para grafos pequenos.